In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
! pip install transformers datasets
! pip install sentencepiece
! pip install transformers[torch]
! pip install xformers

In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_dataset = left_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
left_dataset = left_dataset.train_test_split(test_size=0.2)

In [ ]:
left_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'url', 'date', 'title', 'tags', 'heading', 'source', 'preview_text', 'full_text', 'bias_rating', 'full_text_cleaned'],
        num_rows: 8540
    })
    test: Dataset({
        features: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'url', 'date', 'title', 'tags', 'heading', 'source', 'preview_text', 'full_text', 'bias_rating', 'full_text_cleaned'],
        num_rows: 2136
    })
})

## Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_dataset = left_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2633 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1868 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1610 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1467 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1710 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2006 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1194 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1655 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 32


# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_left_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

## Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import PegasusForCausalLM

model = PegasusForCausalLM.from_pretrained("google/pegasus-large", add_cross_attention=False)
print(model.config.is_decoder, f"{model.__class__} has to be configured as a decoder.")

Some weights of PegasusForCausalLM were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True <class 'transformers.models.pegasus.modeling_pegasus.PegasusForCausalLM'> has to be configured as a decoder.


In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="pegasus_left_fulltext_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=20
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 7.60


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/pegasus_left_fulltext_torch")

## Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/pegasus_left_fulltext_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=50)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

#generator = pipeline("text-generation", model="tobijen/pegasus_left_heading_torch", tokenizer=tokenizer)
#generator(prompt, max_length=20, do_sample=True, temperature=0.2)

social_1  =>  Gay marriage is


[{'generated_text': 'Gay marriage is gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is'}]
education_1  =>  Public education is
[{'generated_text': 'Public education is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is'}]
education_2  =>  Charter schools are
[{'generated_text': 'Charter schools are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are'}]
economic_1  =>  The increase of taxes is
[{'genera

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay gay',
 'social_2': 'Abortion is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is',
 'education_1': 'Public education is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is',
 'education_2': 'Charter schools are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are',
 'economic_1': 'The increase of taxes is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is is 

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "pegasus_left_fulltext": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)